<a href="https://colab.research.google.com/github/LM1997610/Apex_detection/blob/main/YOLOv5/Apex_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO - Obj detection - APEX

reference: [YOLOv5 Transfer Learning](https://kikaben.com/yolov5-transfer-learning-dogs-cats/)



### Get the model:

In [1]:
! git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16413, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 16413 (delta 0), reused 3 (delta 0), pack-reused 16408
Receiving objects: 100% (16413/16413), 14.96 MiB | 29.75 MiB/s, done.
Resolving deltas: 100% (11263/11263), done.


In [5]:
# ! pip -q install -U -r yolov5/requirements.txt

### Prepare data for YOLO

In [3]:
! pip -q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [2]:
import os
import warnings
from PIL import Image
from tqdm import tqdm

from datasets import load_dataset

import tensorflow as tf
from torch import tensor
from torchvision.ops import box_convert

In [3]:
with warnings.catch_warnings():
  warnings.simplefilter("ignore")

  dataset = load_dataset(path='Francesco/apex-videogame')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
for key in dataset:

    dataset[key] = dataset[key].flatten()
    dataset[key] = dataset[key].filter(lambda example: tf.shape(example['objects.bbox'])[0] > 0)

dataset['val'] = dataset.pop('validation')

Filter:   0%|          | 0/2583 [00:00<?, ? examples/s]

Filter:   0%|          | 0/415 [00:00<?, ? examples/s]

Filter:   0%|          | 0/691 [00:00<?, ? examples/s]

Folder structure expected by YOLOv5:

&emsp;&emsp; ├── data \
&emsp;&emsp; ├   └── images \
&emsp;&emsp; │    &emsp;&emsp; ├── train \
&emsp;&emsp; │    &emsp;&emsp; ├── val \
&emsp;&emsp; │    &emsp;&emsp; └── test \
&emsp;&emsp; ├ └── labels \
&emsp;&emsp; │    &emsp;&emsp; ├── train \
&emsp;&emsp; │    &emsp;&emsp; ├── val \
&emsp;&emsp; │    &emsp;&emsp; └── test \


In [7]:
main_dir = 'data/'
os.makedirs(main_dir, exist_ok = True)

sub_dir = ['images/', 'labels/']
for i in sub_dir: os.makedirs(main_dir + i, exist_ok = True)

In [8]:
def convert_to_yolo(boxes):

  image_size = tensor([640, 640])

  boxes[:, :2] /= image_size
  boxes[:, 2:] /= image_size

  boxes = box_convert(boxes, "xywh", "cxcywh")

  return boxes.tolist()

In [9]:
for key in dataset:

  images_dir = main_dir+sub_dir[0]+key; os.makedirs(images_dir, exist_ok = True)
  label_dir = main_dir+sub_dir[1]+key+'/'; os.makedirs(label_dir, exist_ok = True)

  for i in tqdm(range(len(dataset[key]))):
    this_img = dataset[key][i]['image']
    this_labels = dataset[key][i]['objects.category']
    this_boxes = tensor(dataset[key][i]['objects.bbox'],dtype=float)
    this_boxes = convert_to_yolo(this_boxes)
    this_img_name = str(dataset[key][i]['image_id'])

    file_path = os.path.join(images_dir, this_img_name+'.jpg')
    this_img.save(file_path)

    with open(label_dir+this_img_name+'.txt', "w+") as file:
      for line, box in zip(this_labels, this_boxes):
        output = line-1, *box
        output = ' '.join(map(str, output))
        file.write(f"{output}\n")


100%|██████████| 415/415 [00:04<00:00, 100.24it/s]


In [ ]:
# ! rm -r /content/data

## Training:

Load **costum YAML file** to specify the paths to datasets and class definitions for YOLOv5 model.

In [15]:
! git clone https://github.com/LM1997610/Apex_detection
! cp /content/Apex_detection/YOLOv5/costum_file.yaml costum_file.yaml

Cloning into 'Apex_detection'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 153 (delta 32), reused 43 (delta 23), pack-reused 89
Receiving objects: 100% (153/153), 211.88 MiB | 42.01 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [ ]:
! python yolov5/train.py --data costum_file.yaml --weights yolov5s.pt --epochs 15 --batch 64 --freeze 10

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-02-20 13:16:04.287848: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 13:16:04.287894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 13:16:04.289063: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=a_file.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, 

In [ ]:
# ! python yolov5/detect.py --weights best.pt --source apex_game_video.mp4 --device 0